# Pipeline for augmenting the size of training and validation samples

In [ ]:
import os
import numpy as np
from skimage import io
from random import randrange
from skimage.transform import rotate
from skimage.io import imread

### Sorting the labels of the two image directories (lbl + img) together.

In [ ]:
DHM_path = '/Users/behnaz/NCADD/Unet_Cell_body_a2023_v1/cell_body_data/train_data/test/img/'
DHM_all_tif_files = []
N = 0
for file in os.listdir(DHM_path):
    if file.startswith(""):
        if file.endswith(".tif"):
            DHM_all_tif_files.append(file)
            N+=1
        
DHM_all_names = [] 
for file in DHM_all_tif_files: 
    DHM_all_names.append(file[:-4])
   
    
DHM_all_names = sorted(DHM_all_names, key = lambda x: (str(x[10:])))
DHM_all_sorted_tiff_files = [name+'.tif' for name in DHM_all_names]
DHM_all_sorted_directories = [ DHM_path + "/" + file for file in DHM_all_sorted_tiff_files ]


Mask_path = '/Users/behnaz/NCADD/Unet_Cell_body_a2023_v1/cell_body_data/train_data/test/lbl/'
M_all_tiff_files = []
N = 0
for file in os.listdir(Mask_path):
    if file.startswith(""):
        if file.endswith("Masks.tif"):
            M_all_tiff_files.append(file)
            N+=1
        
M_all_names = [] 
for file in M_all_tiff_files: 
    M_all_names.append(file[:-4])
  
    
M_all_names = sorted(M_all_names, key = lambda x: (str(x[10:])))
M_all_sorted_tiff_files = [name+'.tif' for name in M_all_names]
M_all_sorted_directories = [ Mask_path + "/" + file for file in M_all_sorted_tiff_files ]

  
    
pre_all_names = sorted(pre_all_names, key = lambda x: (str(x[10:])))
pre_all_sorted_tiff_files = [name+'.tif' for name in pre_all_names]
pre_all_sorted_directories = [ prediction_path + "/" + file for file in pre_all_sorted_tiff_files ]
for i in range(len(DHM_all_names)):
    print(DHM_all_sorted_directories[i])
    print(M_all_sorted_directories[i])
    print('\n')

### Read images from the specified path, rotate each image by the given angle, and augment them by applying random cropping.

In [ ]:

DHM_save_path = '/Users/behnaz/NCADD/Unet_Cell_body_a2023_v1/cell_body_data/train_data/test/img_patch'

Mask_save_path = '/Users/behnaz/NCADD/Unet_Cell_body_a2023_v1/cell_body_data/train_data/test/lbl_patch'



angles = [0, 15, 30, 45, 60, 75, 90]
matrix = 128
sample = 30


for i in range(len(DHM_all_sorted_directories)):
    DHM_img = imread(str(DHM_all_sorted_directories[i]))
    img = np.array(DHM_img, dtype=np.uint8)
    mask = imread(str(M_all_sorted_directories[i])) 
    mask_img =np.array( mask , dtype=np.uint8)
    mask_img = mask_img>0.99
    for angle in angles:
        DHM_img_rt = rotate(img, angle=angle)
        mask_rt = rotate(mask_img,angle = angle)
        x, y = np.shape(DHM_img_rt)
        for j in range(sample):
            x1 = randrange(0, x - matrix)
            y1 = randrange(0, y - matrix)
            DHM_img_crop = DHM_img_rt[x1:x1 + matrix, y1:y1 + matrix]
            #Convert DHM images to 8-bit image 0-255
            DHM_img_crop_8bit = (DHM_img_crop * 255.0 / DHM_img_crop.max()).astype(np.uint8) 
            mask_crop = mask_rt[x1:x1 + matrix, y1:y1 + matrix]

            
            # Create filenames for the saved images
            dhm_filename = os.path.splitext(os.path.basename(DHM_all_sorted_directories[i]))[0] + "_" + str(j) + "_rt_" + str(angle) + ".tif"
            mask_filename = os.path.splitext(os.path.basename(M_all_sorted_directories[i]))[0] + "_" + str(j) + "_rt_" + str(angle) + ".tif"
            
            # Save images to the specified paths
            io.imsave(os.path.join(DHM_save_path, dhm_filename), DHM_img_crop_8bit)
            io.imsave(os.path.join(Mask_save_path, mask_filename), mask_crop)

   